<a href="https://colab.research.google.com/github/akinolanath5519/Spam-or-Ham-Email-Classifier/blob/main/Email_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#importing the libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from google.colab import files

In [4]:
msg=files.upload()

Saving spam.csv to spam.csv


In [20]:
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [15]:
# Load CSV with proper encoding
df = pd.read_csv("spam.csv", encoding='latin-1')

# Keep only relevant columns
df = df[['v1', 'v2']]

# Rename columns for clarity
df.columns = ['label', 'message']

# Check the first few rows
df.head(5)


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [36]:
#checking the distribution of the label to know if the we have imbalance class
df['label'].value_counts()


,count
label,
ham,4825
spam,747


In [16]:
# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


Applying Text Preprocessing 1


In [24]:
def preprocess_text_tokens(text):
    # 1. Lowercasing
    text = text.lower()
    print("After lowercasing:", text)

    # 2. Remove special characters
    text = re.sub(r'[^a-z\s]', '', text)
    print("After removing special chars:", text)

    # 3. Tokenization
    tokens = nltk.word_tokenize(text)
    print("After tokenization:", tokens)

    # 4. Stop word removal
    tokens_no_stop = [word for word in tokens if word not in stop_words]
    print("After stop word removal:", tokens_no_stop)

    # 5. Lemmatization
    tokens_lemmatized = [lemmatizer.lemmatize(word) for word in tokens_no_stop]
    print("After lemmatization:", tokens_lemmatized)

    return tokens_lemmatized

# Example email
example_email = df['message'][4]  # or df['text'][0] depending on your column
tokens = preprocess_text_tokens(example_email)


After lowercasing: nah i don't think he goes to usf, he lives around here though
After removing special chars: nah i dont think he goes to usf he lives around here though
After tokenization: ['nah', 'i', 'dont', 'think', 'he', 'goes', 'to', 'usf', 'he', 'lives', 'around', 'here', 'though']
After stop word removal: ['nah', 'dont', 'think', 'goes', 'usf', 'lives', 'around', 'though']
After lemmatization: ['nah', 'dont', 'think', 'go', 'usf', 'life', 'around', 'though']


Converting text into vectors using techniques bag of words,TF-IDF

In [27]:
# Function to convert tokens back to string
def tokens_to_string(tokens):
    return ' '.join(tokens)

# Apply preprocessing and convert tokens to string
df['clean_text'] = df['message'].apply(lambda x: tokens_to_string(preprocess_text_tokens(x)))

# # Check
# print(df[['message', 'clean_text']].head(5))


Streaming output truncated to the last 5000 lines.
After lowercasing: \urgent! this is the 2nd attempt to contact u!u have won å£1000call 09071512432 b4 300603t&csbcm4235wc1n3xx.callcost150ppmmobilesvary. maxå£7. 50\""
After removing special chars: urgent this is the nd attempt to contact uu have won call  b tcsbcmwcnxxcallcostppmmobilesvary max 
After tokenization: ['urgent', 'this', 'is', 'the', 'nd', 'attempt', 'to', 'contact', 'uu', 'have', 'won', 'call', 'b', 'tcsbcmwcnxxcallcostppmmobilesvary', 'max']
After stop word removal: ['urgent', 'nd', 'attempt', 'contact', 'uu', 'call', 'b', 'tcsbcmwcnxxcallcostppmmobilesvary', 'max']
After lemmatization: ['urgent', 'nd', 'attempt', 'contact', 'uu', 'call', 'b', 'tcsbcmwcnxxcallcostppmmobilesvary', 'max']
After lowercasing: :( but your not here....
After removing special chars:  but your not here
After tokenization: ['but', 'your', 'not', 'here']
After stop word removal: []
After lemmatization: []
After lowercasing: not directly behind...

In [29]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [31]:
#convert the label to numerical value
# Encode labels to 0 and 1
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['label'])  # spam=1, ham=0

In [34]:
# Split dataset
X_train_text, X_test_text, y_train, y_test = train_test_split(df['clean_text'], y, test_size=0.2, random_state=42)

In [37]:
# BoW vectorizer
bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(X_train_text)
X_test_bow = bow_vectorizer.transform(X_test_text)

# Train Naive Bayes
nb_bow = MultinomialNB()
nb_bow.fit(X_train_bow, y_train)

# Predict and evaluate
y_pred_bow = nb_bow.predict(X_test_bow)
print("=== Bag-of-Words Results ===")
print("Accuracy:", accuracy_score(y_test, y_pred_bow))
print(classification_report(y_test, y_pred_bow))


=== Bag-of-Words Results ===
Accuracy: 0.9802690582959641
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       965
           1       0.96      0.89      0.92       150

    accuracy                           0.98      1115
   macro avg       0.97      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [38]:
# TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)
X_test_tfidf = tfidf_vectorizer.transform(X_test_text)

# Train Naive Bayes
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred_tfidf = nb_tfidf.predict(X_test_tfidf)
print("=== TF-IDF Results ===")
print("Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))


=== TF-IDF Results ===
Accuracy: 0.9668161434977578
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.75      0.86       150

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115



In [39]:
# def predict_message(message, model, vectorizer):
#     # Preprocess the message
#     tokens = preprocess_text_tokens(message)
#     clean_text = ' '.join(tokens)

#     # Convert to vector
#     vect = vectorizer.transform([clean_text])

#     # Predict
#     pred = model.predict(vect)[0]

#     # Convert numeric label back to text
#     label = le.inverse_transform([pred])[0]

#     return label


In [42]:
# while True:
#     msg = input("Enter an email/message (or type 'exit' to stop):\n")
#     if msg.lower() == 'exit':
#         break
#     result = predict_message(msg, nb_bow, bow_vectorizer)
#     print("Prediction:", result)
#     print("-"*40)
